<img src="./101_ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./100_INPUT_OUTPUT.png" width=700px>

Each LLM call is stateless so the LLM sees it for the first time. The more information (context) it has the better will be its response.

We can use this pattern to implement a reflection pattern in stages or we could have gathered the information in another way and passed it in a single request as the first time.


## Agent One is a code expert. 

## Agent Two is an expert code reviewer.

## Agent Three then combines the two to create a final output.

Files `200_code.md`, `200_critique.md` and `200_final.md` are produced rather than outputs here.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
# display_markdown(user_code, raw=True)
# Output below...
with open("200_code.md", "w") as f:
    f.write(user_code)

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

# display_markdown(critique, raw=True)
# Critique displayed below...
with open("200_critique.md", "w") as f:
    f.write(user_code)

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
# display_markdown(essay, raw=True)
# Response to critique displayed below...
with open("200_final.md", "w") as f:
    f.write(user_code)

<br>

### We can of course make this a Class...
